In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt

In [2]:
dip = pd.read_csv('podatki/OpenscienceMetadata.csv',sep='|',encoding='utf_16', on_bad_lines='skip',low_memory=False)

In [20]:
dip

,ID,UDC,Typology,Language,Title,Subtitle,Abstract,Keywords,AlternativeLanguage,AlternativeTitle,AlternativeSubtitle,AlternativeAbstract,AlternativeKeywords,IssueYear,URL,OrganizationID_COBISS,OrganizationID_eVS,COBISSID,NRID
63347,1,2012,http://eprints.gozdis.si/id/eprint/104,1-006,0000033,3486630,10849918,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73359,88701,32,2.20,slv,Mednarodna splošna družboslovna anketa (ISSP) ...,NaN,Mednarodna družboslovna anketa (ISSP) je konti...,mediji;pravica do demonstracij;pravice ekstrem...,eng,NaN,NaN,The International Social Survey Program (ISSP)...,authority;business conditions;civil rights;dem...,2002.0,https://repozitorij.uni-lj.si/IzpisGradiva.php...,3-106,59.0,26843485.0,1468979.0
17594,88702,316.34,2.20,slv,Družbena stratifikacija in mobilnost v jugoslo...,NaN,Namen raziskave je opis in analiza procesov st...,družbena hierarhija;družbene institucije;insti...,eng,NaN,NaN,The purpose of the survey is to describe and a...,hierarchy;social institutions;institutional an...,2002.0,https://repozitorij.uni-lj.si/IzpisGradiva.php...,3-106,59.0,26843741.0,1468980.0
73389,88706,316.334,2.20,slv,"Mednarodna raziskava o političnih vrednotah, 1966",NaN,Študija vključuje vzorce občinskih voditeljev....,mesta;občine;občinske elite;odločanje na ravni...,eng,NaN,NaN,The ICPSR version of this study includes sampl...,cities;communities;community elites;community ...,2001.0,https://repozitorij.uni-lj.si/IzpisGradiva.php...,3-106,59.0,26844765.0,1468984.0
73377,88713,331.5,2.20,slv,"Anketa o delovni sili, 2000/1",NaN,Anketa o delovni sili daje mednarodno primerlj...,sestava gospodinjstva;državljanstvo;država roj...,eng,NaN,NaN,The Labour Force Survey is a unique source of ...,household composition;citizenship;country were...,2001.0,https://repozitorij.uni-lj.si/IzpisGradiva.php...,3-106,59.0,26846557.0,1468991.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93163,5131666,005.336.3,2.11,slv,Izboljšava sistema procesne kontrole polnjenja...,diplomsko delo visokošolskega strokovnega študija,"V Krki, tovarni zdravil, d.d., v obratu za pro...",procesna kontrola;proizvodnja injekcij;kontrol...,eng,IMPROVEMENT OF IN-PROCESS CONTROL SYSTEM FOR F...,NaN,"In Krka d. d., pharmaceutical company, in inje...",Process control,NaN,NaN,NaN,NaN,NaN,NaN
25317,5131667,004.41.05(043.2),2.09,slv,Vpliv mehkih faktorjev na kakovost programske ...,magistrsko delo,V magistrskem delu bomo najprej razvrstili oz....,mehki faktorji;trdi faktorji;kakovost;programs...,eng,Influence of soft factors in software quality,NaN,"Firstly, in this paper we will sort out and ca...",soft factors;hard factors;quality;software;man...,2020.0,https://dk.um.si/IzpisGradiva.php?lang=slv&id=...,3-202,80.0,51735555.0,11943583.0
15279,5131668,331.5,2.11,slv,Zaposlovanje diplomantov Univerze v Novi Gorici,diplomsko delo visokošolskega strokovnega študija,Diplomsko delo obravnava zaposljivost diploman...,diplomanti;izobraževanje;trg dela;,eng,EMPLOYMENT OF GRADUATES OF THE UNIVERSITY OF N...,NaN,The diploma thesis is concerned with the issue...,- graduates,NaN,NaN,NaN,NaN,NaN,NaN
15284,5131671,331.3,2.11,slv,Stres pri ženskah v outsourcing industriji v I...,diplomsko delo visokošolskega strokovnega študija,Outsourcing dela iz razvitih držav v države v ...,outsourcing;klicni centri;stres;,eng,STRESS IN WOMEN IN OUTSOURCING INDUSTRY IN INDIA,NaN,"Nowadays, outsourcing of work from developed c...",outsourcing;call centres;stress;,2009.0,http://dkum.uni-mb.si/Dokument.php?id=8376,3-205,47.0,6443283.0,8715694.0


In [3]:
aut = pd.read_csv('podatki/OpenScienceMetadataAuthors.csv',sep='|',encoding='utf_16', on_bad_lines='warn',low_memory=False)
aut1 = aut.set_index('Name')

In [22]:
aut.groupby("AuthorID").count()

,MetadataID,Name,Surname,Gender
AuthorID,,,,
10,5,5,5,5
11,1,1,1,1
18,1,1,1,1
19,13,13,13,13
21,2,2,2,2
...,...,...,...,...
5033173,1,1,1,1
5033174,1,1,1,1
5033175,1,1,1,1


In [4]:
aut1
print(aut1.loc['Anamarina'].to_string())

MetadataID    1189509
AuthorID       788909
Surname        Ljubić


In [5]:
#https://archive.ics.uci.edu/ml/datasets/Gender+by+Name
#dataset globalnih imen
name = pd.read_csv('podatki/name_gender_dataset.csv',sep=',',encoding='utf_8_sig', on_bad_lines='warn',low_memory=False).drop(['Probability'], axis = 1)
name = name.drop_duplicates(subset=['Name'],keep='first')#ime ki ima vec pojavitev pri dolocenem spolu ostane
name = name.set_index('Name')

In [6]:
m = pd.read_csv('podatki/moska_imena.csv',sep=' ',encoding='cp1250')
z = pd.read_csv('podatki/zenska_imena.csv',sep=' ',encoding='cp1250')

In [7]:
name.loc['A']

Gender    F
Count     2
Name: A, dtype: object

## SLO + en name dataset

In [8]:
mos = m.IME.values
zen = z.IME.values
glob = name.index.values
imena = aut.groupby('Name').count().drop(['MetadataID','AuthorID'], axis = 1).rename(columns={"Surname":"Spol"})
for index, row in imena.iterrows():
    ime = index.replace('-',' ').strip().split(' ')
    if ime[0] in mos:
        imena.at[index,'Spol'] = 'M'
    elif ime[0] in zen:
        imena.at[index,'Spol'] = 'F'
    else:
        if ime[0] in glob:
            imena.at[index,'Spol'] = name.loc[ime[0],"Gender"]
        else:
            imena.at[index,'Spol'] = 'O'

In [9]:
imena
imena.assign(count=0).groupby('Spol').count()
#imena.to_csv('podatki/imena.csv')

,count
Spol,
F,2924
M,3658
O,1482


## Facebook leak

In [10]:
from names_dataset import NameDataset
nd = NameDataset()

In [11]:
print(nd.search("Amal"))

{'first_name': {'country': {'Algeria': 0.036, 'Egypt': 0.313, 'Israel': 0.019, 'Jordan': 0.03, 'Lebanon': 0.027, 'Morocco': 0.293, 'Palestine, State of': 0.023, 'Saudi Arabia': 0.097, 'Syrian Arab Republic': 0.049, 'Tunisia': 0.113}, 'gender': {'Female': 0.924, 'Male': 0.076}, 'rank': {'Algeria': 245, 'Egypt': 104, 'Israel': 143, 'Jordan': 47, 'Lebanon': 26, 'Morocco': 49, 'Palestine, State of': 42, 'Saudi Arabia': 153, 'Syrian Arab Republic': 84, 'Tunisia': 28}}, 'last_name': {'country': {'Algeria': 0.108, 'Egypt': 0.106, 'France': 0.014, 'Iraq': 0.014, 'Italy': 0.015, 'Morocco': 0.503, 'Malaysia': 0.017, 'Saudi Arabia': 0.057, 'Syrian Arab Republic': 0.021, 'Tunisia': 0.146}, 'gender': {}, 'rank': {'Algeria': 336, 'Egypt': 1814, 'France': 6083, 'Iraq': 3934, 'Italy': 13786, 'Morocco': 59, 'Malaysia': 2717, 'Saudi Arabia': 1891, 'Syrian Arab Republic': 1373, 'Tunisia': 136}}}


In [12]:
imena1 = aut.groupby('Name').count().drop(['MetadataID','AuthorID'], axis = 1).rename(columns={"Surname":"Spol"})
for index, row in imena.iterrows():
    ime = index.replace('-',' ').strip().split(' ')
    i = nd.search(ime[0])
    if i['first_name'] != None:
        if "Male" in i['first_name']['gender'] and i['first_name']['gender']['Male']>0.5:
            imena1.at[index,'Spol'] = 'M'
        else:
            imena1.at[index,'Spol'] = 'F'
    else:
        imena1.at[index,'Spol'] = 'O'

In [13]:
imena1
imena1.assign(count=0).groupby('Spol').count()
#imena1.to_csv('podatki/imena1.csv')

,count
Spol,
F,3246
M,4117
O,701


## Facebook + SLO +en dataset

In [14]:
imena2 = aut.groupby('Name').count().drop(['MetadataID','AuthorID'], axis = 1).rename(columns={"Surname":"Spol"})
for index, row in imena.iterrows():
    ime = index.replace('-',' ').strip().split(' ')
    i = nd.search(ime[0])
    if i['first_name'] != None:
        if "Male" in i['first_name']['gender'] and i['first_name']['gender']['Male']>0.5:
            imena2.at[index,'Spol'] = 'M'
        else:
            imena2.at[index,'Spol'] = 'F'
    else:
        if ime[0] in mos:
            imena2.at[index,'Spol'] = 'M'
        elif ime[0] in zen:
            imena2.at[index,'Spol'] = 'F'
        else:
            if ime[0] in glob:
                imena2.at[index,'Spol'] = name.loc[ime[0],"Gender"]
            else:
                imena2.at[index,'Spol'] = 'O'

In [15]:
imena2
imena2.assign(count=0).groupby('Spol').count()
#imena2.to_csv('podatki/imena2.csv')

,count
Spol,
F,3321
M,4230
O,513


## povezava z deli

In [16]:
def gender(x):
    return imena2.loc[x]
aut.assign(Name='n')
aut
aut=aut.dropna(subset=['Name'])
aut['Gender'] = aut['Name'].apply(gender)

In [17]:
dip = dip[dip.ID.apply(lambda x: x.isnumeric())]
dip = dip.astype({"ID": int}).sort_values("ID")

In [18]:
skup = aut.set_index('MetadataID').join(dip.set_index('ID'), how='inner')

In [19]:
#skup.drop(['Abstract','Subtitle','AlternativeTitle','AlternativeSubtitle','AlternativeAbstract'],axis=1).to_csv('podatki/stis_pod.csv')
skup

,AuthorID,Name,Surname,Gender,UDC,Typology,Language,Title,Subtitle,Abstract,...,AlternativeTitle,AlternativeSubtitle,AlternativeAbstract,AlternativeKeywords,IssueYear,URL,OrganizationID_COBISS,OrganizationID_eVS,COBISSID,NRID
88702,967044,Stane,Saksida,M,316.34,2.20,slv,Družbena stratifikacija in mobilnost v jugoslo...,NaN,Namen raziskave je opis in analiza procesov st...,...,NaN,NaN,The purpose of the survey is to describe and a...,hierarchy;social institutions;institutional an...,2002.0,https://repozitorij.uni-lj.si/IzpisGradiva.php...,3-106,59.0,26843741.0,1468980.0
88706,967064,Zdravko,Mlinar,M,316.334,2.20,slv,"Mednarodna raziskava o političnih vrednotah, 1966",NaN,Študija vključuje vzorce občinskih voditeljev....,...,NaN,NaN,The ICPSR version of this study includes sampl...,cities;communities;community elites;community ...,2001.0,https://repozitorij.uni-lj.si/IzpisGradiva.php...,3-106,59.0,26844765.0,1468984.0
88706,967065,Philip,Jacob,M,316.334,2.20,slv,"Mednarodna raziskava o političnih vrednotah, 1966",NaN,Študija vključuje vzorce občinskih voditeljev....,...,NaN,NaN,The ICPSR version of this study includes sampl...,cities;communities;community elites;community ...,2001.0,https://repozitorij.uni-lj.si/IzpisGradiva.php...,3-106,59.0,26844765.0,1468984.0
88713,967096,Zdenka,Blejec,F,331.5,2.20,slv,"Anketa o delovni sili, 2000/1",NaN,Anketa o delovni sili daje mednarodno primerlj...,...,NaN,NaN,The Labour Force Survey is a unique source of ...,household composition;citizenship;country were...,2001.0,https://repozitorij.uni-lj.si/IzpisGradiva.php...,3-106,59.0,26846557.0,1468991.0
88713,967097,Janez,Repinc,M,331.5,2.20,slv,"Anketa o delovni sili, 2000/1",NaN,Anketa o delovni sili daje mednarodno primerlj...,...,NaN,NaN,The Labour Force Survey is a unique source of ...,household composition;citizenship;country were...,2001.0,https://repozitorij.uni-lj.si/IzpisGradiva.php...,3-106,59.0,26846557.0,1468991.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5131668,247992,Franc,Belčič,M,331.5,2.11,slv,Zaposlovanje diplomantov Univerze v Novi Gorici,diplomsko delo visokošolskega strokovnega študija,Diplomsko delo obravnava zaposljivost diploman...,...,EMPLOYMENT OF GRADUATES OF THE UNIVERSITY OF N...,NaN,The diploma thesis is concerned with the issue...,- graduates,NaN,NaN,NaN,NaN,NaN,NaN
5131671,5033176,Valentina,Šurbek,F,331.3,2.11,slv,Stres pri ženskah v outsourcing industriji v I...,diplomsko delo visokošolskega strokovnega študija,Outsourcing dela iz razvitih držav v države v ...,...,STRESS IN WOMEN IN OUTSOURCING INDUSTRY IN INDIA,NaN,"Nowadays, outsourcing of work from developed c...",outsourcing;call centres;stress;,2009.0,http://dkum.uni-mb.si/Dokument.php?id=8376,3-205,47.0,6443283.0,8715694.0
5131671,282164,Marija,Ovsenik,F,331.3,2.11,slv,Stres pri ženskah v outsourcing industriji v I...,diplomsko delo visokošolskega strokovnega študija,Outsourcing dela iz razvitih držav v države v ...,...,STRESS IN WOMEN IN OUTSOURCING INDUSTRY IN INDIA,NaN,"Nowadays, outsourcing of work from developed c...",outsourcing;call centres;stress;,2009.0,http://dkum.uni-mb.si/Dokument.php?id=8376,3-205,47.0,6443283.0,8715694.0
5131672,5033177,Kevin,Žiher,M,004.946+004.76(043.2),2.11,slv,Upravljanje virtualnih okolij in avtomatiziran...,diplomsko delo,Virtualna okolja so vse bolj tesno povezana z ...,...,Managing of virtual machines and automated sof...,NaN,"Virtual environments, which are rapidly develo...",virtualization;virtual machines;Microsoft;Hype...,2020.0,https://dk.um.si/IzpisGradiva.php?lang=slv&id=...,3-202,80.0,51732483.0,12226885.0
